In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
import glob
import matplotlib.pyplot as plt

In [2]:
path = "C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_csf.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_fly_remote.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_jdbc.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_local.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_osb.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_remote_process.csv']

In [3]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)

In [4]:
for df in dfs:
    if df['callType'].iloc[0]=='JDBC' or df['callType'].iloc[0]=='LOCAL':
        df['serviceName'] = df['dsName']
    elif df['callType'].iloc[0]=='RemoteProcess' or df['callType'].iloc[0]=='OSB':
        df['serviceName'] = df['cmdb_id']

In [5]:
# df.head()

In [5]:
df = pd.concat(dfs, axis=0, ignore_index=True)
# df.head()

In [6]:
df = df.drop('dsName', axis=1, index=None)
# df = df.drop(['path'], axis=1)
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,CSF,1590422400042,3212.0,True,fe171a0d9608272247dd,fe1718e7207827a247dd,fe171ae35818272247dd,os_021,csf_001
1,CSF,1590422400043,521.0,True,fe171b1dc723053247dd,fe1712efc05305a247dd,fe171260ff33053247dd,os_022,csf_001
2,CSF,1590422400043,2483.0,True,fe1710044423683247dd,fe171c71bc7368b247dd,fe1711b3b433683247dd,os_021,csf_001
3,CSF,1590422400044,2230.0,True,fe1717f21d28273247dd,fe1712d8e85827c247dd,fe17174eec38273247dd,os_021,csf_001
4,CSF,1590422400044,2482.0,True,fe171972b143684247dd,fe171f20ea9368c247dd,fe1719218f53684247dd,os_021,csf_001


In [7]:
anomaly_time = 1590430140000
# 1590430140000 db 007
# 1590431940000 docker 008
# 1590433740000 os 21
df = df[(df.startTime >= (anomaly_time-1200000)) & (df.startTime <= (anomaly_time + 60000))]
# df.to_csv(path + 'trace_data_os18.csv')


In [8]:
df

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
262802,CSF,1590428940026,2235.0,True,0f17142adb09611fe314,0f171e05877961afe314,0f1711a1dc19611fe314,os_021,csf_001
262803,CSF,1590428940026,1470.0,True,0f1715be0d29613fe314,0f171cd4505961afe314,0f171a473c39613fe314,os_021,csf_001
262804,CSF,1590428940027,372.0,True,0f17153a5d85962fe314,0f171399d51696bfe314,0f171475d595962fe314,os_022,csf_001
262805,CSF,1590428940027,2533.0,True,0f1712bc9c44034fe314,0f1719bfa79403bfe314,0f171f47d754034fe314,os_021,csf_001
262806,CSF,1590428940028,169.0,True,0f1719b75a49614fe314,0f171f51b99961cfe314,0f17112f1559614fe314,os_021,csf_001
...,...,...,...,...,...,...,...,...,...
9105944,RemoteProcess,1590430172262,19.0,True,0f171da30e0301b59045,0f1719faf5378866c045,0f1711e769696560c045,docker_005,docker_005
9105945,RemoteProcess,1590430172263,32.0,True,0f171c9f61630197b045,0f1715845b578876c045,0f171a6cf6107570c045,docker_005,docker_005
9105946,RemoteProcess,1590430172367,20.0,True,0f171c9f61630197b045,0f17178f438988fcc045,0f17140b602775bcc045,docker_005,docker_005
9105947,RemoteProcess,1590430172460,19.0,True,0f171c9f61630197b045,0f171beedc7198c2d045,0f171e88982975a2d045,docker_005,docker_005


In [9]:
df1 = df[df['callType']=='RemoteProcess']
df1 = df1[['pid','cmdb_id']]
df1 = df1.set_index('pid')

In [10]:
elapse_time = {}
children = {}

def do_thing(row):
    if row['id'] in df1.index:
        row['serviceName'] = df1.at[row['id'],'cmdb_id']
    if row['pid'] != 'None':
        children[row['pid']] = children.get(row['pid'], [])
        children[row['pid']].append(row['id'])
    elapse_time[row['id']] = float(row['elapsedTime'])
    return row
df = df.apply(do_thing, axis=1)


df['actual_time'] = 0.0

def get_actual_time(row):
    total_child = 0.0
    if row['id'] in children:
        for child in children[row['id']]:
            total_child += elapse_time[child]
    row['actual_time'] = row['elapsedTime'] - total_child
    return row

df = df.apply(get_actual_time, axis = 1)

df = df[~(df['serviceName'].str.contains('csf', na=True))]

In [27]:
df = df.sort_values(['traceId', 'startTime'], ignore_index=True)

In [28]:
df.head(60)

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,actual_time
0,OSB,1590430283976,1339.0,True,0f1710004746968c0c55,0f171611d056968c0c55,None,os_022,os_022,15.0
1,CSF,1590430283983,1324.0,True,0f1710004746968c0c55,0f171922ce7696fc0c55,0f171611d056968c0c55,os_022,docker_002,11.0
2,RemoteProcess,1590430283993,1313.0,True,0f1710004746968c0c55,0f17158d5739399d0c55,0f171922ce7696fc0c55,docker_002,docker_002,238.0
3,FlyRemote,1590430284100,4.0,True,0f1710004746968c0c55,0f17134f5a8249441c55,0f17158d5739399d0c55,docker_002,fly_remote_001,4.0
4,CSF,1590430284105,287.0,True,0f1710004746968c0c55,0f171c76965349941c55,0f17158d5739399d0c55,docker_002,docker_007,253.0
5,RemoteProcess,1590430284195,34.0,True,0f1710004746968c0c55,0f171654d620633a1c55,0f171c76965349941c55,docker_007,docker_007,27.0
6,LOCAL,1590430284216,7.0,True,0f1710004746968c0c55,0f1717ce4842638b1c55,0f171654d620633a1c55,docker_007,db_003,2.0
7,JDBC,1590430284216,3.0,True,0f1710004746968c0c55,0f17197dba72638b1c55,0f1717ce4842638b1c55,docker_007,db_003,3.0
8,JDBC,1590430284220,2.0,True,0f1710004746968c0c55,0f171cb8c70363cb1c55,0f1717ce4842638b1c55,docker_007,db_003,2.0
9,CSF,1590430284394,201.0,True,0f1710004746968c0c55,0f1717fae75749a62c55,0f17158d5739399d0c55,docker_002,docker_007,176.0


In [13]:
df.to_csv('trace526209_.csv')

In [ ]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows()):
    if row['id'] in csf_cmdb:
        df.at[index, 'serviceName'] = csf_cmdb[row['id']]

In [ ]:
elapse_time = {}
children = {}
for index, row in tqdm(df.iterrows()):
    if row['pid'] != 'None':
        if row['pid'] in children.keys():
            children[row['pid']].append(row['id'])
        else:
            children[row['pid']] = [row['id']]
    elapse_time[row['id']] = float(row['elapsedTime'])

In [ ]:
df['actual_time'] = 0.0
for index, row in tqdm(df.iterrows()):
    total_child = 0.0
    if row['id'] not in children.keys():
        df.at[index, 'actual_time'] = row['elapsedTime']
        continue
    for child in children[row['id']]:
        total_child += elapse_time[child]
    df.at[index, 'actual_time'] = row['elapsedTime'] - total_child

In [ ]:
df = df.sort_values(['traceId','startTime','callType'], ignore_index = True)

In [ ]:
df.serviceName.unique()

In [ ]:
df.head(60)

In [20]:
grouped_df = df.groupby(['cmdb_id', 'serviceName'])[['startTime','actual_time']]

In [21]:
grouped_df.groups.keys()

dict_keys([('docker_001', 'db_007'), ('docker_001', 'db_009'), ('docker_001', 'docker_001'), ('docker_001', 'docker_007'), ('docker_001', 'docker_008'), ('docker_001', 'fly_remote_001'), ('docker_002', 'db_007'), ('docker_002', 'db_009'), ('docker_002', 'docker_002'), ('docker_002', 'docker_007'), ('docker_002', 'docker_008'), ('docker_002', 'fly_remote_001'), ('docker_003', 'db_007'), ('docker_003', 'db_009'), ('docker_003', 'docker_003'), ('docker_003', 'docker_005'), ('docker_003', 'fly_remote_001'), ('docker_004', 'db_007'), ('docker_004', 'db_009'), ('docker_004', 'docker_004'), ('docker_004', 'docker_005'), ('docker_004', 'fly_remote_001'), ('docker_005', 'db_003'), ('docker_005', 'docker_005'), ('docker_007', 'db_003'), ('docker_007', 'docker_007'), ('docker_008', 'db_003'), ('docker_008', 'docker_008'), ('os_021', 'docker_003'), ('os_021', 'docker_004'), ('os_021', 'os_021'), ('os_022', 'docker_001'), ('os_022', 'docker_002'), ('os_022', 'os_022')])

In [13]:
def esd_test_statistics(x, hybrid=True):
  """
  Compute the location and dispersion sample statistics used to carry out the ESD test.
  """
  if hybrid:
    location = pd.Series(x).median(skipna=True) # Median
    dispersion = np.median(np.abs(x - np.median(x))) # Median Absolute Deviation
  else:  
    location = pd.Series(x).mean(skipna=True) # Mean
    dispersion = pd.Series(x).std(skipna=True) # Standard Deviation
    
  return location, dispersion    

In [14]:
# def seasonal_mean(x, freq):
#   """
#   Returns the mean of the timeseries for each period in x.
  
#   x    : List, array, or series containing the time series
#   freq : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)
#   """
#   return np.array([pd.Series(x[i::freq]).mean(skipna=True) for i in range(freq)])

def ts_S_Md_decomposition(x):
  """
  Decomposes the timeseries using a modified STL method:Rx = X - Sx - X^~.
    Rx : Residuals
    X  : Original time series
    Sx : Seasonality component (average value per period)
    X^~: Median of original timeseries
    
  x    : List, array, or series containing the time series 
  freq : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)  
  """
  nobs = len(x)
  
  # Seasonality
#   period_averages = seasonal_mean(x, freq)
#   seasonal = np.tile(period_averages, nobs // freq + 1)[:nobs]
  
  # Median
  med = np.tile(pd.Series(x).median(skipna=True), nobs)
  
  # Residuals
#   res = np.array(x) - seasonal - med
  res = np.array(x) - med
  return {"observed": np.array(x), "median":med, "residual":res}

In [15]:
def esd_test(x, alpha=0.95, ub=0.499, hybrid=True):
  """
  Carries out the Extreme Studentized Deviate(ESD) test which can be used to detect one or more outliers present in the timeseries
  
  x      : List, array, or series containing the time series
  freq   : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)
  alpha  : Confidence level in detecting outliers
  ub     : Upper bound on the fraction of datapoints which can be labeled as outliers (<=0.499)
  hybrid : Whether to use the robust statistics (median, median absolute error) or the non-robust versions (mean, standard deviation) to test for anomalies
  """
  nobs = len(x)
  if ub > 0.4999:
    ub = 0.499
  k = max(int(np.floor(ub * nobs)), 1) # Maximum number of anomalies. At least 1 anomaly must be tested.
#   res_tmp = ts_S_Md_decomposition(x)["residual"] # Residuals from time series decomposition
    
  # Carry out the esd test k times  
  res = np.ma.array(x, mask=False) # The "ma" structure allows masking of values to exclude the elements from any calculation
  anomalies = [] # returns the indices of the found anomalies
  for i in range(1, k+1):
    location, dispersion = esd_test_statistics(res, hybrid) # Sample statistics
    tmp = np.abs(res - location) / dispersion
    idx = np.argmax(tmp) # Index of the test statistic
    test_statistic = tmp[idx] 
    n = nobs - res.mask.sum() # sums  nonmasked values
    critical_value = (n - i) * t.ppf(alpha, n - i - 1) / np.sqrt((n - i - 1 + np.power(t.ppf(alpha, n - i - 1), 2)) * (n - i - 1)) 
    if test_statistic > critical_value:
      anomalies.append(test_statistic)
    res.mask[idx] = True  
  return np.mean(anomalies)

In [16]:
grouped_df.groups.keys()

NameError: name 'grouped_df' is not defined

In [22]:
class RCA():
    def local_initiate(self):
        self.dockers = ['docker_001', 'docker_002', 'docker_003', 'docker_004',
                'docker_005', 'docker_006', 'docker_007', 'docker_008']
        self.docker_hosts = ['os_017', 'os_018', 'os_019', 'os_020']

        self.docker_kpi_names = ['container_cpu_used', None]
        self.os_kpi_names = ['Sent_queue', 'Received_queue']
        self.db_kpi_names = ['Proc_User_Used_Pct','Proc_Used_Pct','Sess_Connect','On_Off_State', 'tnsping_result_time']

        self.docker_lookup_table = {}
        for i in range(len(self.dockers)):
            self.docker_lookup_table[self.dockers[i]] = self.docker_hosts[i % 4]
            
    def find_anomalous_rows(self, min_threshold = 5):
        table = self.anomaly_chart.copy()
        threshold = max( 0.5 * table.stack().max(), min_threshold)
        dodgy_rows = []
        just_rows = []
        for column in table:
            v = 0
            r = ''
            for index, row in table.iterrows():
                if (row[column] > threshold):
                    if index == column:
                        dodgy_rows.append([index, row[column]])
                        just_rows.append(index)
                        break
                    elif (row[column] > v):
                        v = row[column]
                        r = index
            if r != '':
                dodgy_rows.append([r, column, v])
                just_rows.append(r)
        
        output = self.localize(dodgy_rows, list(set(just_rows)))
        return output


    def find_anomalous_kpi(self, cmdb_id):
        kpi_names = []
        if 'os' in cmdb_id:
            kpi_names = self.os_kpi_names
        elif 'docker' in cmdb_id:
            kpi_names = self.docker_kpi_names
        else:
            kpi_names = self.db_kpi_names

        return kpi_names


    def localize(self, dodgy_rows, just_rows):
        n = len(just_rows)
        if n < 1:
            return None
        if n == 1:
            KPIs = self.find_anomalous_kpi(just_rows[0])
            to_be_sent = []
            for KPI in KPIs:
                to_be_sent.append([just_rows[0], KPI])
            return to_be_sent
        if n == 2:
            r0 = just_rows[0]
            r1 = just_rows[1]
            if ('os' in r0) and ('os' in r1):
                KPI = self.find_anomalous_kpi('os_001')
                return [['os_001', KPI[0]],['os_001', KPI[1]]]
            elif ('docker' in r0) and ('docker' in r1):
                if self.docker_lookup_table[r0] == self.docker_lookup_table[r1]:
                    KPI = self.find_anomalous_kpi(self.docker_lookup_table[r0])
                    return [[self.docker_lookup_table[r0], KPI]]
            else:
                KPI0s = self.find_anomalous_kpi(r0)
                KPI1s = self.find_anomalous_kpi(r1)
                to_be_sent = []
                for kpi in KPI0s:
                    to_be_sent.append([r0, kpi])
                for kpi in KPI1s:
                    to_be_sent.append([r1, kpi])
                return to_be_sent
        if n > 2:
            dodgy_rows.sort(key = lambda x: x[2], reverse = True)
            just_rows = [x[0] for x in dodgy_rows]
            just_rows = list(set(just_rows))
            return self.localize(dodgy_rows[:2], just_rows[:2])

In [19]:
rca = RCA()
rca.local_initiate()
rca.anomaly_chart = anomaly_chart
output = rca.find_anomalous_rows()

NameError: name 'anomaly_chart' is not defined

In [ ]:
output

In [23]:
anomaly_chart = pd.DataFrame()
for (a, b), value in grouped_df:
    value['time_group'] = value.startTime//60000
    value =value.groupby(['time_group'])['actual_time'].mean().reset_index()
    result = esd_test(value['actual_time'].to_numpy(), alpha=0.95, ub=0.02, hybrid=True)
    anomaly_chart.loc[b,a] = result
    # print(a,b,' = ',len(result)/len(value)*100)

In [24]:
anomaly_chart.sort_index(inplace=True)
anomaly_chart

,docker_001,docker_002,docker_003,docker_004,docker_005,docker_007,docker_008,os_021,os_022
db_003,NaN,NaN,NaN,NaN,2.218662,8.426948,3.427777,NaN,NaN
db_007,2.451175,3.113193,3.353311,2.929689,NaN,NaN,NaN,NaN,NaN
db_009,3.935336,5.031015,3.147331,4.003738,NaN,NaN,NaN,NaN,NaN
docker_001,2.592030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.687135
docker_002,NaN,3.642814,NaN,NaN,NaN,NaN,NaN,NaN,7.402834
docker_003,NaN,NaN,2.917402,NaN,NaN,NaN,NaN,4.504866,NaN
docker_004,NaN,NaN,NaN,2.229114,NaN,NaN,NaN,4.241102,NaN
docker_005,NaN,NaN,2.971381,4.093297,2.750001,NaN,NaN,NaN,NaN
docker_007,3.787066,4.032622,NaN,NaN,NaN,2.809472,NaN,NaN,NaN
docker_008,3.717222,3.508460,NaN,NaN,NaN,NaN,3.148636,NaN,NaN


In [ ]:
test = grouped_df.get_group(('docker_008', 'docker_008'))

In [ ]:
x = test[['startTime','actual_time']]
# x_val = x['actual_time'].to_numpy()
# x

In [ ]:
x['time_group'] = x.startTime//10000
x = x.groupby(['time_group'])['actual_time'].mean().reset_index()

In [ ]:
x_val = x['actual_time'].to_numpy()

In [ ]:
result = esd_test(x_val, freq=3, alpha=0.99, ub=0.4, hybrid=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pd.Series(x_val).index, x_val, color="blue", label = "Original")
ax.scatter(result, x_val[result], color='red', label='Anomaly')
plt.legend(loc="best")
plt.show()

In [ ]:
x = test[['startTime','actual_time']]
# x_val = x['actual_time'].to_numpy()
# x
x['time_group'] = x.startTime//1000
x = x.groupby(['time_group'])['actual_time'].mean().reset_index()

In [ ]:
x_test = x.copy()
x_test.index = pd.to_datetime(x_test['time_group'], unit="s")

In [ ]:
x_test